In [2]:
import os
os.chdir('../')
%pwd

'/home/dheeraj/Projects/Explainable-AI-Non-EEG'

In [17]:
from src.dataset.create_dataset import DataframePrepAllMod  
from src.constants.constants import *
from src.model.model import ConvModel, train_cp
from src.utils.cross_val import TrainTestSplitter
from src.utils.utils import *
import tensorflow as tf


In [4]:
import argparse
import glob
import json
import os

In [5]:
def main(args):
    device = "GPU" if tf.config.list_physical_devices('GPU') else "CPU"
    num_epochs = args.e
    batch_size = args.batch
    
    # Check if dataset file exists and create it if not
    if not os.path.isfile(os.path.join(SAVED_DATASET_DIR, args.f)): 
        print("Dataset file not found!")
        print("Generating dataset file:", args.f)
        create_data_set = DataframePrepAllMod(
            subjects=SUBJECTS, modals=MODALS, modals_1=MODALS_1, modals_2=MODALS_2,
            df_hr_spo2=HR_SPO2_FILE_PATH, df_acc_temp_eda=ACC_TEMP_EDA_FILE_PATH,
            label=LABELS, frame_rate=60, max_size=360*8
        )
        data_set = create_data_set.dataframe_prep_allmod()
        data_set.to_json(os.path.join(SAVED_DATASET_DIR, args.f))

    else:
        print("Dataset file found!")
        data_set = pd.read_json(os.path.join(SAVED_DATASET_DIR, args.f))
        if args.cross_val == 'LOSO':
            train_df, test_df = TrainTestSplitter.train_test_loso(data_set, subject_index=args.subject_index)
            train_filename = f"train_data_{args.cross_val}_{args.subject_index}.pkl"
            test_filename = f"test_data_{args.cross_val}_{args.subject_index}.pkl"

        else:
            train_df, test_df = TrainTestSplitter.train_test_losego(data_set, seg_index=args.seg_index, num_seg=args.num_seg)
            train_filename = f"train_data_{args.cross_val}_{args.seg_index}_{args.num_seg}.pkl"
            test_filename = f"test_data_{args.cross_val}_{args.seg_index}_{args.num_seg}.pkl"

        train_path = os.path.join(SAVED_DATASET_DIR, train_filename)            
        test_path = os.path.join(SAVED_DATASET_DIR, test_filename)

        save_dataframe_to_pickle(train_df, train_path) 
        save_dataframe_to_pickle(test_df, test_path)

    # Prepare data for training and validation 
    train_data, train_label = dataframe_to_array(train_df)
    train_data, train_label, val_data, val_label = preprocess_train_val(train_data, train_label)
    print("train set size:", len(train_data))
    print("val set size:", len(val_data))
    

    # Model training
    input_shape = train_data.shape[1:]

    if args.cross_val == 'LOSO':
        model_path = os.path.join(SAVED_MODEL_DIR, f"model_{args.cross_val}_{args.subject_index}.h5")
    else:
        model_path = os.path.join(SAVED_MODEL_DIR, f"model_{args.cross_val}_{args.seg_index}_{args.num_seg}.h5")

    model = ConvModel(input_shape=input_shape, nb_classes=len(LABELS))
    optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
    train_cp(model, optimizer, train_data, train_label, val_data, val_label, epochs=args.e, batch_size=args.batch, model_path=model_path)

        





SyntaxError: invalid syntax (1517588486.py, line 6)

In [24]:
def train_cp(model, x_train, y_train, x_val, y_val, epochs, batch_size, path):
    """
    Trains the ConvModel model.

    Parameters:
    - model (keras.Model): ConvModel model.
    - x_train (np.ndarray): Training data.
    - y_train (np.ndarray): Training labels.
    - x_val (np.ndarray): Validation data.
    - y_val (np.ndarray): Validation labels.
    - epochs (int): Number of epochs.
    - batch_size (int): Batch size.
    - path (str): Path to save the model checkpoints.

    Returns:
    - keras.callbacks.History: History of the training process.
    """
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=path, 
                                                                monitor='val_accuracy', save_weights_only=True, 
                                                                mode='max', save_best_only=True)
    
    print('Training...')

    history = model.model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size, 
                              callbacks=[lr_scheduler, early_stopping, model_checkpoint_callback])

    # Save the model checkpoints
    if not os.path.exists(path):
        os.makedirs(path)
    
    model.model.save(path)

    print('Model saved')

In [7]:
ROOT_DIR = os.path.join(os.getcwd())
SAVE_DATASET_DIR = os.path.join(ROOT_DIR, 'data', 'created_dataset', 'processed_dataframe_allmod_8Hz.pkl')
loaded_df = load_dataframe_from_pickle(SAVE_DATASET_DIR)

In [8]:
cross_val = TrainTestSplitter(subjects=SUBJECTS, labels=LABELS)
train_df, test_df = cross_val.train_test_loso(df=loaded_df, subject_index=0)

In [9]:
train_data, train_label = dataframe_to_array(train_df)
train_data, train_label, val_data, val_label = preprocess_train_val(train_data, train_label)
    

In [23]:
input_shape = train_data.shape[1:]
model = ConvModel(input_shape=input_shape, nb_classes=len(LABELS))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [19]:
SAVED_MODEL_DIR = os.path.join(ROOT_DIR, 'saved_models')

In [25]:
model_path = os.path.join(SAVED_MODEL_DIR, f"model_LOSO_0.h5")

In [31]:
train_label

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [30]:
train_data.shape, train_label.shape, val_data.shape, val_label.shape

((4468, 60, 7, 1), (4468, 4), (1916, 60, 7, 1), (1916, 4))

In [28]:
train_cp(model, train_data, train_label, val_data, val_label, epochs=100, batch_size=64, path=model_path)

Training...
Epoch 1/100


ValueError: in user code:

    File "/home/dheeraj/miniconda3/envs/xaieeg/lib/python3.10/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/home/dheeraj/miniconda3/envs/xaieeg/lib/python3.10/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/dheeraj/miniconda3/envs/xaieeg/lib/python3.10/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/home/dheeraj/miniconda3/envs/xaieeg/lib/python3.10/site-packages/keras/src/engine/training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/dheeraj/miniconda3/envs/xaieeg/lib/python3.10/site-packages/keras/src/engine/training.py", line 1185, in compute_loss
        return self.compiled_loss(
    File "/home/dheeraj/miniconda3/envs/

In [ ]:
if __name__ == "__main__":
    
    #for i in os.listdir('data/raw'):
        #if i == "fully_connected_patches_scale_diff_all_shapes__15_kps.pkl":
    test_num = 1
    test_depth = 2
    #test_shapes = "2048, 2048" #"256, 256" # 4096 best  
    dataset_to_train = "fully_connected_patches_scale_diff.pkl"
    #graph_file_to_test = i
    # not required for now
    #test_graph_type = 'fully_connected'
    #test_node_attributes = ["norm_descriptors"] #"pos, norm_pos, size, norm_size, angle, norm_angle, norm_descriptors, class_id, quadrant ,patches"
    #test_edge_attribute = 'scale_diff' #"distance, scale_diff, angle_diff"
    #test_descriptor_type = 'hu' 
    
    parser = argparse.ArgumentParser(description='Training configuration')
    parser.add_argument('--f', type=str, default=dataset_to_train, help="train dataset file name")
    parser.add_argument('--m', type=str, default=f'ConvModel-{dataset_to_train}', help="model name")
    parser.add_argument('--device', type=str, default='cuda', help="cuda / cpu")
    parser.add_argument('--batch', type=int, default=32, help="batch size")
    parser.add_argument('--e', type=int, default=100, help="number of epochs")
    parser.add_argument('--lr', type=float, default=1e-3, help="learning rate")#1e-3, 5.e-3
    parser.add_argument('--optimizer', type=str, default='Adam', help="optimizer")#1e-3, 5.e-3
    parser.add_argument('--model_dir', type=str, default="models/", help="path to save model")
    parser.add_argument('--cross_val', type=str, default='LOSO', help="cross validation method (LOSO / LOSegO)")
    parser.add_argument('--subject_index', type=int, default=0, help="subject index for LOSO")
    parser.add_argument('--num_seg', type=int, default=1, help="number of segments to leave for LOSegO")
    parser.add_argument('--segment_index', type=int, default=0, help="segment index for LOSegO")
    #parser.add_argument('--n_train', type=int, default=140, help="number of samples for train set")
    #parser.add_argument('--n_test', type=int, default=140, help="number of samples for test set")
    #parser.add_argument('--depth', type=int, default=test_depth, help="depth of encoder and decoder")

    #writer = SummaryWriter()
    #writer.add_text('Arguments', json.dumps(vars(args), indent=4))
    main(args)
    #writer.close()
    print("Training Finished, training next model now")

In [ ]:
df_train, df_test = train_test_losego(df)
data = np.array([np.array(xi, dtype='float32') for xi in df_train['Data']])
#data = data.mean(axis=1)

cpdir = os.path.join("checkpoints", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
checkpoint_filepath = cpdir
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, 
                                                            monitor='val_accuracy', save_weights_only=True, 
                                                            mode='max', save_best_only=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor=0.5, patience=10, min_lr=0.0001) 

early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=0, mode='auto',
                                        baseline=None, restore_best_weights=False)

moddir = os.path.join("saved_model_new/All_modalities/LoSego/8Hz-60s/Seq_seg/", 
                    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.3, random_state=1, shuffle=True)

batch_size = 64 #min(x_train.shape[0]/10, 16)    

nb_classes = len(np.unique(y_val))
y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_val = (y_val - y_val.min())/(y_val.max()-y_val.min())*(nb_classes-1)   

Y_train = keras.utils.to_categorical(y_train, nb_classes)
Y_val = keras.utils.to_categorical(y_val, nb_classes)
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std)
x_val = (x_val - x_train_mean)/(x_train_std)
x_train = x_train.reshape(x_train.shape + (1,))
x_val = x_val.reshape(x_val.shape + (1,))
x_val_std = x_val.std()
x_val_mean = x_val.mean()

In [7]:
from dataclasses import dataclass

In [ ]:
@dataclass
class TrainingConfig:
    PATH: str = os.path